In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [3]:
device = 'cpu'
np.random.seed(0)

dataset = GermanDataset(device=device, sensitive_feature_labels=["Age", "Sex"])

In [4]:
from time import time
start = time()

In [5]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI_fc = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    _, _, EI_kde = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    _, _, EI_fb = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD, EI_fc, EI_kde, EI_fb

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [6]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.005, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 300
SGD_hp_test['batch_size'] = 256
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 1.1

In [7]:
kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 300/300 [00:02<00:00, 101.51epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:02<00:00, 107.11epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:02<00:00, 104.70epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:02<00:00, 102.70epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          dp         eo      eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ---------  --------
         0.0001          0           0.6775            0.71   0.166667   0.00805195  0.0221286  0.037037
         0.001           0           0.7575            0.79   0.0333333  0.1         0.0183562  0.230159
         0.005           0           0.7775            0.805  0.0525526  0.131667    0.0272216  0.232804
         0.01            0           0.77125           0.805  0.0909091  0.135       0.0409203  0.222222
         0.1             0           0.76125           0.805  0.0906977  0.168333    0.0579557  0.272487


The learning rate is decided as 0.005.


In [8]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.001
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [9]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.001]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EI_hp_test['fairness'] = 'EI'

kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 300/300 [00:06<00:00, 45.00epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:06<00:00, 47.45epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:07<00:00, 40.05epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:07<00:00, 40.68epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
          0.001       0              0.7575            0.79   0.0333333  0.1        0.0183562  0.230159
          0.001       0.2            0.74375           0.77   0          0.0666667  0.0189041  0.121693
          0.001       0.6            0.70625           0.74   0          0.02       0.0136986  0.037037
          0.001       0.8            0.68875           0.735  0          0.025      0.0136986  0.0396825
          0.001       0.9            0.41625           0.385  0.0407679  0.111667   0.135406   0.135406
          0.001       0.95           0.33125           0.315  0.0194743  0.0583333  0.0821918  0.0821918


The lambda for EI is decided as 0.9.

In [10]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.001
EI_hp_kde['lambda_'] = 0.6

In [11]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.001]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95, 0.999]
EI_hp_test['fairness'] = 'EI'

fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 300/300 [00:04<00:00, 73.70epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:03<00:00, 77.33epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:03<00:00, 81.89epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:03<00:00, 84.27epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          dp          eo      eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ----------  --------
          0.001      0               0.7575            0.79   0.0333333  0.1         0.0183562   0.230159
          0.001      0.6             0.75625           0.79   0.0333333  0.1         0.0183562   0.230159
          0.001      0.8             0.75625           0.79   0          0.0933333   0.00520548  0.201058
          0.001      0.9             0.75375           0.785  0          0.0916667   0.0120548   0.190476
          0.001      0.95            0.75125           0.78   0          0.0966667   0.0120548   0.208995
          0.001      0.999           0.705             0.735  0          0.00201299  0.0187917   0.037037


In [12]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.001
EI_hp_fc['lambda_'] = 0.95

In [13]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.001]
EI_hp_test['lambda_'] = [0, 0.2, 0.5, 0.6, 0.8, 0.9,]
EI_hp_test['fairness'] = 'EI'

fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 300/300 [00:04<00:00, 66.31epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:04<00:00, 67.37epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:04<00:00, 70.65epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 300/300 [00:04<00:00, 71.67epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ---------  ---------
          0.001        0              0.7575           0.79   0.0333333  0.1         0.0183562  0.230159
          0.001        0.2            0.745            0.765  0          0.065       0.0170355  0.111111
          0.001        0.5            0.6875           0.71   0          0.00896104  0.0170355  0.0185185
          0.001        0.6            0.69             0.73   0          0.00666667  0.0187917  0.0187917
          0.001        0.8            0.695            0.735  1          0.00798701  0          0.0185185
          0.001        0.9            0.6875           0.73   0          0           0          0


In [14]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.001
EI_hp_fb['lambda_'] = 0.8

#### Model training

In [15]:
seeds = np.arange(1,6)

SGD, EI_fc, EI_kde, EI_fb = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds)

/var/folders/_1/grmv219n3mzbyfj882m03j3c0000gn/T/ipykernel_3495/3561611485.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 300/300 [00:02<00:00, 106.77epochs/s]


training seed 2 started


Training: 100%|██████████| 300/300 [00:02<00:00, 104.06epochs/s]


...

training seed 4 started


Training: 100%|██████████| 300/300 [00:04<00:00, 69.74epochs/s]


training seed 5 started


Training: 100%|██████████| 300/300 [00:04<00:00, 68.76epochs/s]

Training finished for all seeds.


In [16]:
result = []
models = ["SGD", "EI FC", "EI KDE", "EI FB"]
sol = [SGD, EI_fc, EI_kde, EI_fb]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean     ei_var    dp_mean      dp_var    eo_mean     eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ---------  ---------  ---------  ----------  ---------  ---------  -----------  ----------
SGD                0.773      0.00509902  0.118163   0.0657833  0.0803232  0.0153933   0.0866929  0.024348     0.099715    0.0115914
EI FC              0.775      0.00774597  0.0739149  0.0168262  0.0883681  0.00680762  0.0925325  0.0185392    0.102092    0.0132455
EI KDE             0.727      0.0334066   0.2        0.4        0.0346554  0.0171508   0.0372249  0.0230019    0.0523739   0.0162579
EI FB              0.705      0.0176068   0.4        0.489898   0.0151493  0.00541723  0.0166334  0.0102015    0.0382006   0.0230774


In [17]:
end = time() - start
print("Time: {:.7f} ms".format(end * 1000))
print("Time (s): {:.7f} s".format(end))

Time: 189609.7111702 ms
Time (s): 189.6097112 s


In [18]:
188.1042340 % 60

8.104233999999991